In [613]:

import math

import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import v2

from models import Patchify

batch_size = 128
model_dim = 64
max_seq_len = 256

transform = v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])
data = datasets.MNIST(root="data", train=False, download=True, transform=transform)
dl = DataLoader(data, batch_size=batch_size, shuffle=False)
len(data)
    


10000

In [614]:
for batch, (images, labels) in enumerate(dl):
    # batch size of 128 yields 78 batches from 10_000 test images
    print(f"Batch {batch}: images.shape = {images.shape}, labels.shape = {labels.shape}")
    break

# get first batch only for play
x, y = next(iter(dl))
x.shape

Batch 0: images.shape = torch.Size([128, 1, 28, 28]), labels.shape = torch.Size([128])


torch.Size([128, 1, 28, 28])

In [615]:

patchify = Patchify(patch_size=7, model_dim=model_dim)
x = patchify(x)
x.shape  # 128x1x28x28 -> 128x16x64

torch.Size([128, 16, 64])

In [616]:
# build simplest/smallest positional encoding from scratch
pe = torch.zeros(max_seq_len, model_dim)
pe.shape
# note that pe.shape here is one image as 16 patches in 64 dims, i.e. same as one item from the patch batch


torch.Size([256, 64])

In [617]:
position = torch.arange(0, max_seq_len, dtype=torch.float)
position  # 0, 1, 2, ..., 255

tensor([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,  11.,
         12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.,
         24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,  33.,  34.,  35.,
         36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,  44.,  45.,  46.,  47.,
         48.,  49.,  50.,  51.,  52.,  53.,  54.,  55.,  56.,  57.,  58.,  59.,
         60.,  61.,  62.,  63.,  64.,  65.,  66.,  67.,  68.,  69.,  70.,  71.,
         72.,  73.,  74.,  75.,  76.,  77.,  78.,  79.,  80.,  81.,  82.,  83.,
         84.,  85.,  86.,  87.,  88.,  89.,  90.,  91.,  92.,  93.,  94.,  95.,
         96.,  97.,  98.,  99., 100., 101., 102., 103., 104., 105., 106., 107.,
        108., 109., 110., 111., 112., 113., 114., 115., 116., 117., 118., 119.,
        120., 121., 122., 123., 124., 125., 126., 127., 128., 129., 130., 131.,
        132., 133., 134., 135., 136., 137., 138., 139., 140., 141., 142., 143.,
        144., 145., 146., 147., 148., 14

In [618]:
# turn each position int into its own row
position = position.unsqueeze(1)
position.shape

torch.Size([256, 1])

In [619]:
magic = math.log(10_000.0) / model_dim  # 10_000 is a magic number worked out by Vaswani et al.
magic

0.14391156831212787

In [620]:
# we want to alternate between adding sine and cosine values to even/odd indices
even_positions = torch.arange(0, model_dim, 2)
even_positions.shape

torch.Size([32])

In [621]:
# apply magic scalar to div tensor
div_terms = even_positions * - magic
div_terms

tensor([-0.0000, -0.2878, -0.5756, -0.8635, -1.1513, -1.4391, -1.7269, -2.0148,
        -2.3026, -2.5904, -2.8782, -3.1661, -3.4539, -3.7417, -4.0295, -4.3173,
        -4.6052, -4.8930, -5.1808, -5.4686, -5.7565, -6.0443, -6.3321, -6.6199,
        -6.9078, -7.1956, -7.4834, -7.7712, -8.0590, -8.3469, -8.6347, -8.9225])

In [622]:
# here we essentially compute the denominator for the sine/cosine functions (10000^(2i/model_dim))
div_terms = torch.exp(div_terms)
div_terms  # shape: 32

tensor([1.0000e+00, 7.4989e-01, 5.6234e-01, 4.2170e-01, 3.1623e-01, 2.3714e-01,
        1.7783e-01, 1.3335e-01, 1.0000e-01, 7.4989e-02, 5.6234e-02, 4.2170e-02,
        3.1623e-02, 2.3714e-02, 1.7783e-02, 1.3335e-02, 1.0000e-02, 7.4989e-03,
        5.6234e-03, 4.2170e-03, 3.1623e-03, 2.3714e-03, 1.7783e-03, 1.3335e-03,
        1.0000e-03, 7.4989e-04, 5.6234e-04, 4.2170e-04, 3.1623e-04, 2.3714e-04,
        1.7783e-04, 1.3335e-04])

In [623]:
# here we make use of 'broadcasting' to apply each position int (0, 1, 2...) as a scalar factor to div_terms
broadcast = position * div_terms  # 256x1 * 32 -> 256x32
broadcast

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [1.0000e+00, 7.4989e-01, 5.6234e-01,  ..., 2.3714e-04, 1.7783e-04,
         1.3335e-04],
        [2.0000e+00, 1.4998e+00, 1.1247e+00,  ..., 4.7427e-04, 3.5566e-04,
         2.6670e-04],
        ...,
        [2.5300e+02, 1.8972e+02, 1.4227e+02,  ..., 5.9996e-02, 4.4990e-02,
         3.3738e-02],
        [2.5400e+02, 1.9047e+02, 1.4283e+02,  ..., 6.0233e-02, 4.5168e-02,
         3.3871e-02],
        [2.5500e+02, 1.9122e+02, 1.4340e+02,  ..., 6.0470e-02, 4.5346e-02,
         3.4005e-02]])

In [624]:
# pe is a 256x64 tensor of zeroes - now we fill it with alternating sine/cosine of our (pos/10000^(2i/d)) values
pe[:, 0::2] = torch.sin(broadcast)  # skips down the evens in every column
pe[:, 1::2] = torch.cos(broadcast)  # skips down the odds in every column
pe[255, :]

tensor([-0.5064, -0.8623,  0.4024, -0.9155, -0.8985,  0.4391,  0.6582,  0.7528,
        -0.8641,  0.5033, -0.7031, -0.7111,  0.9787,  0.2055,  0.5250, -0.8511,
         0.3591,  0.9333,  0.2694,  0.9630,  0.9796, -0.2012, -0.9708, -0.2399,
         0.9781, -0.2083, -0.2340,  0.9722, -0.9842, -0.1768, -0.2560, -0.9667,
         0.5577, -0.8301,  0.9423, -0.3348,  0.9907,  0.1364,  0.8797,  0.4754,
         0.7218,  0.6921,  0.5685,  0.8227,  0.4381,  0.8989,  0.3335,  0.9427,
         0.2522,  0.9677,  0.1901,  0.9818,  0.1429,  0.9897,  0.1073,  0.9942,
         0.0806,  0.9968,  0.0604,  0.9982,  0.0453,  0.9990,  0.0340,  0.9994])

In [625]:
pe = pe.unsqueeze(0)  # add batch dimension to broadcast over any number of batches on forward()
pe.shape  # 1x256x64

torch.Size([1, 256, 64])

In [626]:
# recall that x (our first batch of patched images) has shape 128x16x64
x.size(1)

16

In [627]:
# so we need to trim the pe to match the number of patches in x (has to be < max_seq_len)
slice = pe[:, : x.size(1)]
slice.shape

torch.Size([1, 16, 64])

In [628]:
x[0, 5, :]  # 5th patch of first image in batch

tensor([ 0.0765,  0.1969, -0.1129, -0.0251,  0.3599, -0.4023, -0.4207,  0.1494,
        -0.1252, -0.2122, -0.1672, -0.1167,  0.2257,  0.0744,  0.1566, -0.4740,
        -0.0221, -0.1182,  0.7875, -0.6767,  0.4351,  0.0106, -0.1075,  0.0433,
         0.0736, -0.1243, -0.1440,  0.0702, -0.1675, -0.1359, -0.0190,  0.1608,
        -0.7834,  0.1120,  0.1835, -0.1378,  0.3011, -0.3472,  0.0826,  0.1269,
         0.5378, -0.3999,  0.0614,  0.2722, -0.2307,  0.1115, -0.2543,  0.0290,
         0.3140, -0.0544, -0.0722, -0.1867, -0.0401,  0.3119,  0.0558, -0.1538,
         0.2347,  0.0710,  0.2733,  0.3991,  0.2501, -0.0445,  0.0229, -0.3157],
       grad_fn=<SliceBackward0>)

In [629]:
# and finally we broadcast the pe across each of our 16 patches in every batch
# this is what is returned by PositionalEncoding.forward(x)
x = x + slice
x[0, 5, :]

tensor([-0.8825,  0.4805, -0.6840, -0.8460,  0.6838, -1.3484,  0.4382, -0.3627,
         0.8748, -0.2225,  0.7596,  0.2590,  1.0022,  0.7045,  0.7751,  0.3118,
         0.4573,  0.7594,  1.1537,  0.2539,  0.7125,  0.9713,  0.1018,  1.0211,
         0.2310,  0.8633, -0.0257,  1.0632, -0.0787,  0.8601,  0.0476,  1.1586,
        -0.7334,  1.1108,  0.2210,  0.8615,  0.3293,  0.6525,  0.1036,  1.1267,
         0.5536,  0.6000,  0.0733,  1.2721, -0.2219,  1.1115, -0.2476,  1.0290,
         0.3190,  0.9456, -0.0684,  0.8133, -0.0373,  1.3119,  0.0580,  0.8462,
         0.2363,  1.0710,  0.2744,  1.3991,  0.2510,  0.9555,  0.0236,  0.6843],
       grad_fn=<SliceBackward0>)